# Tracery and Python

by [Allison Parrish](http://www.decontextualize.com/)

This is a tutorial on how to use [Tracery](http://www.crystalcodepalace.com/tracery.html) in Python.
Tracery is a computer language for random text generation originally developed
by [Kate Compton](https://twitter.com/galaxykate).

The easiest way to use Tracery in Python is to install [pytracery](https://github.com/aparrish/pytracery), my Python port of Kate's original code. You can install it on the command line with `pip`:

    $ pip install tracery

(You may need to type `sudo` in front of `pip`: `sudo pip install tracery`.) If you're running this in Jupyter Notebook, you can execute the cell below:

In [30]:
!pip install tracery

Then you need to import the `tracery` library:

In [2]:
import tracery

You don't need Python to use Tracery! [Here's a version of this tutorial](http://air.decontextualize.com/tracery/) that you can use with any implementation of Tracery. I recommend Beau Gunderson's [Tracery writer](https://beaugunderson.com/tracery-writer/) as a kind of Tracery playground. You can also use Kate Compton's
[Tracery tutorial](http://www.crystalcodepalace.com/traceryTut.html), which has a [visual editor](http://www.brightspiral.com/tracery/) or [Cheap Bots Done Quick](http://cheapbotsdonequick.com/), which has a built-in editor for writing Tracery grammars for Twitter bots with a minimum amount of fuss.

You might be interested in reading [Nora Reed's explanation of how
@nerdgarbagebot works](http://barrl.net/2801), which takes you through the
process of ideating and implementing a Tracery grammar for a Twitter bot. (Nora
Reed makes a lot of amazing bots with Tracery, including
[@thinkpiecebot](https://twitter.com/thinkpiecebot).)

## Rules and expansions

A Tracery *grammar* is a series of rules that tell the computer how to put text
together, piece by piece. Tracery grammars consist of a series of *rules* and
*expansions*. The goal of writing a Tracery grammar is to write rules and
expansions that, when followed by the computer, produce interesting (funny,
insightful, poetic) text. The word for generating a text from a grammar is
"expand"---we'll be talking a lot below about "expanding" the grammar into a
text. (Hopefully the reasons for using this word will become clear!)

In Python, Tracery rules and expansions are written as dictionaries, where the rules are keys and the expansions are values. Here's an example of a complete, but very boring, Tracery grammar:

In [3]:
rules = {
  "origin": "Hello, world!"
}

To generate text from this grammar, first create a Tracery `Grammar` object like so, passing the rules as the only parameter:

In [4]:
grammar = tracery.Grammar(rules)

Then call the `.flatten()` method of the `Grammar` object with `"#origin#"` as the only parameter. (I'll talk about what `#origin#` means in a second.)

In [5]:
grammar.flatten("#origin#")

'Hello, world!'

This grammar can produce only one text: `Hello, world!`. Not very interesting,
but helpful for the moment to illustrate how a grammar is put together and how to make it produce some output.

Here's a Tracery grammar with two rules, written again as a dictionary, where each rule and its expansion are key/value pairs:

In [6]:
rules = {
  "origin": "Hello, #noun#!",
  "noun": "galaxy"
}
grammar = tracery.Grammar(rules)
grammar.flatten("#origin#")

'Hello, galaxy!'

This grammar, again, can only ever produce one text: `Hello, galaxy!` But it
accomplishes it in a slightly more sophisticated way. Notice in the expansion
for the `origin` rule the following text:

    #noun#

When the Tracery generator encounters text that looks like this---a word surrounded by
`#` signs---it looks in the grammar for a rule with the same name as the word,
and *replaces* the text with the expansion for that rule.

Let's add a third rule to this grammar, just to see how it looks:

In [9]:
rules = {
  "origin": "#greeting#, #noun##punctuation#",
  "greeting": "Howdy",
  "noun": "galaxy",
  "punctuation": "?"
}
grammar = tracery.Grammar(rules)
grammar.flatten("#origin#")

'Howdy, galaxy?'

> EXERCISE: Add another rule for the punctuation at the end of the sentence, so
> that the grammar produces the text "Howdy, galaxy?"

## Adding alternatives

The examples above are really boring, because they can only ever produce one
output. In order for a grammar to be able to produce different outputs, we need
to make the expansions of our rules have *alternatives* for the computer to
choose between. Rules with alternatives look like this:

    "rule": ["alternative one", "alternative two", "alternative three"]

That is: the value of the rule is a *list* of strings (instead of an individual string). When Tracery expands a rule whose value is a list, it will select one item from the list at random.

Here's our "Hello, world!" grammar, now with multiple alternatives for what
we're greeting:

In [10]:
rules = {
  "origin": "#greeting#, #noun#!",
  "greeting": "Howdy",
  "noun": ["world", "solar system", "galaxy", "local cluster", "universe"]
}
grammar = tracery.Grammar(rules)
grammar.flatten("#origin#")

'Howdy, universe!'

Run the cell over and over again and you'll see different outputs. (Sometimes
it'll look like it isn't working, but that's just because the computer randomly
selected the same alternative twice in a row. It can happen!)

Let's make this "Hello, world!" example *even more interesting* by adding
alternatives for the `greeting` rule:

In [11]:
rules = {
  "origin": "#greeting#, #noun#!",
  "greeting": ["Howdy", "Hello", "Greetings", "What's up", "Hey", "Hi"],
  "noun": ["world", "solar system", "galaxy", "local cluster", "universe"]
}
grammar = tracery.Grammar(rules)
grammar.flatten("#origin#")

'Hey, solar system!'

Sometimes for debugging purposes, it's nice to generate multiple outputs from the same grammar in one cell execution. To do this, `print()` the value of the `.flatten()` function in a `for` loop. (You don't have to re-create the `Grammar` object each time.)

In [12]:
rules = {
  "origin": "#greeting#, #noun#!",
  "greeting": ["Howdy", "Hello", "Greetings", "What's up", "Hey", "Hi"],
  "noun": ["world", "solar system", "galaxy", "local cluster", "universe"]
}
grammar = tracery.Grammar(rules)
for i in range(5):
    print(grammar.flatten("#origin#"))

Greetings, universe!
Hey, world!
What's up, galaxy!
Greetings, solar system!
Howdy, universe!


Remember that in Python you can format dictionaries and lists with some flexibility. For example, your grammar might be a bit more readable if you write each option on a separate line:

In [13]:
rules = {
  "origin": "#greeting#, #noun#!",
  "greeting": [
    "Howdy",
    "Hello",
    "Greetings",
    "What's up",
    "Hey",
    "Hi"
  ],
  "noun": [
    "world",
    "solar system",
    "galaxy",
    "local cluster",
    "universe"
  ]
}
grammar = tracery.Grammar(rules)
grammar.flatten("#origin#")

'Hello, galaxy!'

You don't always have to write the expansions as string literals and list literals. You can use a variable with a list assigned to it, for example—this is especially helpful if you have a long list of things that you plan to use in multiple grammars, or if you want to get the list of things from another source (e.g., a text file).

## Modifiers

Let's make a more sophisticated grammar that produces sentences in the format
"Dammit Jim, I'm a X, not a Y!" popularized by the ground-breaking science
fiction program, *Star Trek*. I happen to have a list of professions, which I'm going to put into a variable here. (I got this list from Darius Kazemi's [Corpora Project](https://github.com/dariusk/corpora/tree/master/data)—an excellent place to find lists of things. And they're already preformatted in a way that makes it easy to cut-and-paste them into your Tracery grammars.)

In [16]:
professions = [
    "accountant",
    "actor",
    "archeologist",
    "astronomer",
    "audiologist",
    "bartender",
    "curator",
    "detective",
    "economist",
    "editor",
    "engineer",
    "epidemiologist",
    "farmer",
    "flight attendant",
    "forest fire prevention specialist",
    "graphic designer",
    "hydrologist",
    "librarian",
    "mathematician",
    "middle school teacher",
    "nutritionist",
    "painter",
    "rancher",
    "referee",
    "reporter",
    "sailor",
    "sociologist",
    "stonemason",
    "surgeon",
    "tailor",
    "taxi driver",
    "teacher",
    "therapist",
    "tour guide",
    "umpire",
    "undertaker",
    "urban planner",
    "veterinarian",
    "web developer",
    "welder",
    "writer",
    "zoologist"
]

The grammar for generating our Star Trek phrase might look like this:

In [25]:
rules = {
  "origin": "#interjection#, #name#! I'm a #profession#, not a #profession#!",
  "interjection": ["alas", "congratulations", "eureka", "fiddlesticks",
    "good grief", "hallelujah", "oops", "rats", "thanks", "whoa", "yes"],
  "name": ["Jim", "John", "Tom", "Steve", "Kevin", "Gary", "George", "Larry"],
  "profession": professions
}
grammar = tracery.Grammar(rules)
grammar.flatten("#origin#")

"eureka, Jim! I'm a referee, not a economist!"

This is pretty good, but there are problems. The first is that we typed in all
of the interjections in lower case, but they're supposed to have the first
letter capitalized (since they're at the beginning of the sentence). The second
problem is that the grammar occasionally produces something like

    yes, George! I'm a economist, not a zoologist!

"A economist" isn't right. It should be "a*n* economist." English indefinite
articles are tricky that way!

There are several ways to solve these problems. We could just change all of our
interjections to be capitalized, and add the appropriate article to the
beginning of each profession. But (1) this will be time consuming and (2) it
means that we won't ever be able to *re-use* those same rules with the
unmodified versions of those rules. What to do?

Thankfully, Tracery comes equipped with a series of *modifiers* that take the expansion of a rule and apply a transformation to it. The modifiers are included with pytracery, but they're in a separate module, so you need to import them in their own import statement:

In [26]:
from tracery.modifiers import base_english

And then you have to explicitly "add" them to the `Grammar` object after you create it, like so:

    grammar = tracery.Grammar(rules)
    grammar.add_modifiers(base_english)

The two modifiers we're going to use are `.a`, which adds the appropriate indefinite article before the expansion of a rule, and `.capitalize`, which capitalizes the first letter of the expansion.

Use the modifers by adding `.a` inside the `#` signs, *right after* the name of the rule. For example, change:

    #interjection#

to

    #interjection.capitalize#

Here's our "Dammit Jim" generator with the modifiers in place:

In [27]:
rules = {
  "origin": "#interjection.capitalize#, #name#! I'm #profession.a#, not #profession.a#!",
  "interjection": ["alas", "congratulations", "eureka", "fiddlesticks",
    "good grief", "hallelujah", "oops", "rats", "thanks", "whoa", "yes"],
  "name": ["Jim", "John", "Tom", "Steve", "Kevin", "Gary", "George", "Larry"],
  "profession": professions
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.flatten("#origin#")

"Alas, Gary! I'm an engineer, not a taxi driver!"

Nice! Another modifier you can use is `.s`, which turns the text in the expansion into its plural version. Using this, we can modify the above example to be a *Star Wars* meme instead of a *Star Trek* one:

In [29]:
rules = {
  "origin": "These aren't the #profession.s# we're looking for.",
  "profession": professions
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.flatten("#origin#")

"These aren't the archeologists we're looking for."

## The `origin` rule

By convention, the "starting" rule of Tracery grammars is called `origin`. A lot of tools that use Tracery grammars follow this convention, and for ease of interoperability it's probably best if you do too. But you can actually use any name you want, as long as you use that name in the call to `.flatten()`. For example, we could rewrite the above example like so:

In [31]:
rules = {
  "origin": "These aren't the #profession.s# we're looking for.",
  "profession": professions
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
grammar.flatten("#origin#")

"These aren't the reporters we're looking for."

Like any other rule, the "starting" rule can have multiple options. We could use this to, for example, create a grammar that outputs Star Wars memes half the time and Star Trek memes the other half:

In [32]:
rules = {
  "origin": ["#interjection.capitalize#, #name#! I'm #profession.a#, not #profession.a#!",
             "These aren't the #profession.s# we're looking for."],
  "interjection": ["alas", "congratulations", "eureka", "fiddlesticks",
    "good grief", "hallelujah", "oops", "rats", "thanks", "whoa", "yes"],
  "name": ["Jim", "John", "Tom", "Steve", "Kevin", "Gary", "George", "Larry"],
  "profession": professions
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
for i in range(10):
    print(grammar.flatten("#origin#"))

Yes, Steve! I'm a stonemason, not a hydrologist!
Whoa, Kevin! I'm a librarian, not a therapist!
These aren't the hydrologists we're looking for.
Thanks, Tom! I'm a rancher, not a welder!
Whoa, Larry! I'm an astronomer, not a detective!
These aren't the accountants we're looking for.
These aren't the editors we're looking for.
These aren't the engineers we're looking for.
These aren't the surgeons we're looking for.
Rats, Larry! I'm a taxi driver, not an archeologist!


## Rules within rules within rules

The grammars we've written together so far have replacement syntax (`#somethinglikethis#`) only in the expansions for the `origin` rule. But you can include that syntax in any expansion you want! This is a powerful tool for building sophisticated grammars that are built up from reusable parts. For example, this tiny model of English reuses the `noun` and `verb` rules in multiple places, thereby preventing repetition and increasing expressiveness.

In [35]:
rules = {
    "origin": "#nounphrase.capitalize# #verbphrase#.",
    "nounphrase": ["the #noun#", "the #noun#", "#noun.a#", "#noun.a#", "the #noun# that #verbphrase#",
                   "the #noun# #prep# #nounphrase#"],
    "verbphrase": ["#verb#", "#verb# #nounphrase#", "#verb# #prep# #nounphrase#"],
    "noun": ["amoeba", "dichotomy", "seagull", "trombone", "corsage", "restaurant", "suburb"],
    "verb": ["awakens", "bends", "burns", "closes", "expands", "fails", "fractures", "gathers",
             "melts", "opens", "ripens", "scatters", "stops", "sways", "turns", "unfurls", "worries"],
#note: ergative verbs (like the ones above) do not need an object and thus can be used transivitely and intransitively
    "prep": ["in", "on", "over", "against"]
}
grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
for i in range(10):
    print(grammar.flatten("#origin#"))

A dichotomy fails a suburb.
The suburb stops.
The amoeba that fails ripens.
The amoeba that ripens worries.
The seagull turns.
The corsage that unfurls gathers.
The restaurant opens.
A restaurant fractures the amoeba.
An amoeba unfurls.
The trombone worries the amoeba against the corsage that bends the suburb over the amoeba.


## Next steps

Congratulations, you now know the basics of writing a Tracery grammar and how to use them in Python.

Tracery has a number of features that we didn't go into here, including the
ability to *save* the output of a rule to be re-used later in the same
expansion. See [Kate Compton's tutorial](http://cheapbotsdonequick.com/)
for more information. You might be interested in [these advanced text generators](http://www.brightspiral.com/) that Kate Compton made with Tracery.

If you're a Javascript programmer and you want to incorporate Tracery into your
own projects, [the source code is available
here](https://github.com/galaxykate/tracery) (also available as [a Node module](https://github.com/v21/tracery)).

In [36]:
!pip install pycorpora --install-option="--corpora-zip-url=https://github.com/dariusk/corpora/archive/master.zip"


/usr/local/lib/python3.6/site-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Skipping bdist_wheel for pycorpora, due to binaries being disabled for it.
  Running setup.py install for pycorpora ... done


The following is a brief tutorial for the Pycorpora project which may be useful

In [72]:
# just the keys
list(divination.zodiac["western_zodiac"].keys())

['Aries',
 'Taurus',
 'Gemini',
 'Cancer',
 'Leo',
 'Virgo',
 'Libra',
 'Scorpio',
 'Sagittarius',
 'Capricorn',
 'Aquarius',
 'Pisces']

AttributeError: 'list' object has no attribute 'keys'

In [60]:
sign = random.choice(list(divination.zodiac["western_zodiac"].keys()))

In [61]:
sign

'Cancer'

In [75]:
divination.zodiac["western_zodiac"][sign]['gloss']

'The Fish'

In [81]:
from pycorpora import archetypes
archetypes.character['characters'][0]['qualities']

['brave', 'willing', 'determined']

In [84]:
archetypes.character['characters'][0].keys()

dict_keys(['name', 'synonyms', 'qualities', 'nature'])

In [91]:
archetypes.character.keys()

dict_keys(['description', 'characters'])

In [104]:
list(archetypes.character['characters'])

[{'name': 'hero',
  'synonyms': ['protagonist'],
  'qualities': ['brave', 'willing', 'determined'],
  'nature': 'good'},
 {'name': 'anti-hero',
  'synonyms': ['antagonist'],
  'qualities': ['unwilling', 'reluctant'],
  'nature': 'neutral'},
 {'name': 'analyst',
  'synonyms': ['reasoner'],
  'qualities': ['rational', 'logical'],
  'nature': 'neutral'},
 {'name': 'bureaucrat',
  'synonyms': ['lawyer', 'politician', 'manager'],
  'qualities': ['obedient'],
  'nature': 'neutral'},
 {'name': 'caretaker',
  'synonyms': ['healer', 'doctor', 'nurse'],
  'qualities': ['caring', 'healing'],
  'nature': 'good'},
 {'name': 'catalyst',
  'synonyms': ['doer'],
  'qualities': ['dynamic', 'changeable', 'mercurial', 'unpredictable'],
  'nature': 'neutral'},
 {'name': 'coward',
  'synonyms': ['wimp'],
  'qualities': ['weak', 'craven', 'fearful'],
  'nature': 'evil'},
 {'name': 'curmudgeon',
  'synonyms': ['grump'],
  'qualities': ['opinionated', 'cranky', 'resolute'],
  'nature': 'evil'},
 {'name': 'men

In [65]:
# if this were javascript, you could: divination.zodiac.western_zodiac[sign].gloss

In [67]:
symbols = []
for sign in list(divination.zodiac["western_zodiac"].keys()):
    symbol = divination.zodiac["western_zodiac"][sign]['unicode_symbol']
    symbols.append(symbol)

In [86]:
symbols

['♈', '♉', '♊', '♋', '♌', '♍', '♎', '♏', '♐', '♑', '♒', '♓']

In [117]:
archetype_names = []
for hero in archetypes.character['characters']:
#    print(hero)
    archetype_names.append(hero['name'])
#     archetype_name = archetypes.character['characters'][hero]['name']
#     archetype_name.append(archetype_names)

In [118]:
archetype_names

['hero',
 'anti-hero',
 'analyst',
 'bureaucrat',
 'caretaker',
 'catalyst',
 'coward',
 'curmudgeon',
 'mentor',
 'shaman',
 'herald',
 'trickster',
 'fool',
 'child',
 'self',
 'hermit',
 'wanderer',
 'hunter',
 'judge',
 'weaver',
 'death',
 'giver',
 'superhuman',
 'dreamer',
 'gossip',
 'guardian',
 'hedonist',
 'leader',
 'martyr',
 'everyman',
 'poet',
 'rebel',
 'slave',
 'samaritan',
 'scholar',
 'survivor',
 'sycophant',
 'temptress',
 'tyrant',
 'victim',
 'waif',
 'minion']

In [128]:
archetype_qualities = []
for hero in archetypes.character['characters']:
#    print(hero)
#    archetype_qualities.append(hero['qualities'])
    archetype_qualities.append(', '.join(hero['qualities']))

In [129]:
archetype_qualities

['brave, willing, determined',
 'unwilling, reluctant',
 'rational, logical',
 'obedient',
 'caring, healing',
 'dynamic, changeable, mercurial, unpredictable',
 'weak, craven, fearful',
 'opinionated, cranky, resolute',
 'educating, guiding, helpful',
 'mystical, mysterious',
 'expositional, revealing',
 'confusing, egocentric, misleading, clever, manipulative',
 'thoughtless, silly, amusing',
 'innocent, naive, simple, straightforward, credulous, endearing',
 'reflecting',
 'quiet, secretive',
 'curious',
 'curious, dedicated, driven',
 'fair, unbiased, patient',
 'thorough, crafty',
 'final, morbid, dark, sombre',
 'giving, generous, charitable, patronizing',
 'incredible',
 'wishful, ambitious',
 'talkative, expositional, unfettered, egocentric',
 'strong, protective',
 'egocentric, short-sighted',
 'inspiring, compelling',
 'self-sacrificing, noble',
 'common, typical, average, normal',
 'thoughtful, lyrical, myterious, enlightening',
 'rebellious, questioning',
 'trapped, abused,

In [69]:
# shorter way of writing it
symbols = []
for sign, info in divination.zodiac["western_zodiac"].items():
    symbol = info['unicode_symbol']
    symbols.append(symbol)

In [70]:
sign_to_gloss_map = {}
for sign, info in divination.zodiac["western_zodiac"].items():
    sign_to_gloss_map[sign] = info['gloss']
sign_to_gloss_map

{'Aries': 'The Ram',
 'Taurus': 'The Bull',
 'Gemini': 'The Twins',
 'Cancer': 'The Crab',
 'Leo': 'The Lion',
 'Virgo': 'The Maiden',
 'Libra': 'The Scales',
 'Scorpio': 'The Scorpion',
 'Sagittarius': 'The Archer',
 'Capricorn': 'The Mountain Sea-goat',
 'Aquarius': 'The Water-bearer',
 'Pisces': 'The Fish'}

In [120]:
sign_to_gloss_map['Virgo']

'The Maiden'

The following is an attempt to generate new zodiac signs using Tracery and PyCorpora

In [247]:
from pycorpora import archetypes
from pycorpora import divination
from pycorpora import mythology
from pycorpora import corporations
from pycorpora import science
from pycorpora import plants
from pycorpora import objects

archetype_qualities = []
archetype_names = []
monster_names = []
creatures = []
first_date = []
last_date = []
companies = []
industries = []
elements = []
cannabis = []
stuff = []

for hero in archetypes.character['characters']:
#    print(hero)
    archetype_qualities.append(', '.join(hero['qualities']))
    
for hero in archetypes.character['characters']:
#    print(hero)
    archetype_names.append(hero['name'])
    
for monster in mythology.greek_monsters['greek_monsters']:
#    print(monster)
    monster_names.append(monster)    
    
for monster in mythology.monsters['names']:
    #print(monster)
    creatures.append(monster)  
    
for plant in plants.cannabis['cannabis']:
    #print(plant)
    cannabis.append(plant)    
    
for i in range(1,15):    
 #   print(i)
    first_date.append(str(i))
        
for i in range(15,32):    
#    print(i)
    last_date.append(str(i))
    
for item in corporations.fortune500['companies']:
    companies.append(item)

for item in corporations.industries['industries']:
    industries.append(item)  

for item in objects.objects['objects']:
    stuff.append(item)      
    
for i in range(118):
    symbol = science.elements['elements'][i]['name']
    elements.append(symbol)
    

In [274]:
rules = {
    "qualities":archetype_qualities,
    "gloss":archetype_names,
    "monsters":monster_names,
    "character":creatures,
    "stuff": stuff,
    "weekday":["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    "month":["January","February","March","April","May","June","July","August","September","October","November","Decemeber"],
    "first_date": first_date,
    "last_date": last_date,
    "companies": companies,
    "industries": industries,
    "elements":elements,
    "cannabis":cannabis,
    "old_elements":["Fire", "Earth", "Air", "Water", "Metal", "Aether"],
    "condition":["You were born on a #weekday#", "You were born between #month# #first_date# - #last_date#"],
    
    "origin":"""Astrology Generator\n --------- \nCondition: #condition#   \nSign: #monsters# the #gloss.capitalize# -- \nSymbol: #character.capitalize# -- \nElement: #old_elements# -- \nQualities: #qualities# """
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
print(grammar.flatten("#origin#"))

Astrology Generator
 --------- 
Condition: You were born on a Wednesday   
Sign: Chimera the Child -- 
Symbol: Selkie -- 
Element: Fire -- 
Qualities: innocent, naive, simple, straightforward, credulous, endearing 


In [284]:
rules = {
    "qualities":archetype_qualities,
    "gloss":archetype_names,
    "monsters":monster_names,
    "character":creatures,
    "stuff": stuff,
    "weekday":["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    "month":["January","February","March","April","May","June","July","August","September","October","November","Decemeber"],
    "first_date": first_date,
    "last_date": last_date,
    "companies": companies,
    "industries": industries,
    "elements":elements,
    "cannabis":cannabis,
    "destructive":["destroy","dismantle","smash","ruin", "sabotage","obliterate"],
    "condition":["You were born on a #weekday#", "You were born between #month# #first_date# - #last_date#"],
    
    "origin":"{Vaguely Anti-Corporatists/Millenial Astrology} \n --------- \nCondition: #condition#   \nSign: #monsters# the #gloss.capitalize# -- \nSymbol: #stuff.capitalize.a.capitalize# -- \nQualities: #qualities# -- \nElement: #elements# -- \nPower Cannabis Strain: #cannabis# -- \nQuest: Use #industries# to #destructive# #companies#"
}

grammar = tracery.Grammar(rules)
grammar.add_modifiers(base_english)
print(grammar.flatten("#origin#"))

{Vaguely Anti-Corporatists/Millenial Astrology} 
 --------- 
Condition: You were born between July 5 - 19   
Sign: Telekhines the Hedonist -- 
Symbol: A Canvas -- 
Qualities: caring, healing -- 
Element: Zinc -- 
Power Cannabis Strain: Yumboldt -- 
Quest: Use Farming to obliterate US Foods, Inc.
